In [2]:
import pandas as pd
import numpy as np
import random
from collections import Counter
import matplotlib.pyplot as plt
import time
import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.grid_search import GridSearchCV
import sklearn.preprocessing
import sklearn.cross_validation
import xgboost as xgb

In [ ]:
# PREPARE DATA
data = pd.read_csv('train.csv').set_index("ID")
test = pd.read_csv('test.csv').set_index("ID")


In [ ]:
print data.shape
print test.shape

In [ ]:
# remove constants
nunique = pd.Series([data[col].nunique() for col in data.columns], index = data.columns)
constants = nunique[nunique<2].index.tolist()
data = data.drop(constants,axis=1)
test = test.drop(constants,axis=1)



['VAR_0008', 'VAR_0009', 'VAR_0010', 'VAR_0011', 'VAR_0012', 'VAR_0018', 'VAR_0019', 'VAR_0020', 'VAR_0021', 'VAR_0022', 'VAR_0023', 'VAR_0024', 'VAR_0025', 'VAR_0026', 'VAR_0027', 'VAR_0028', 'VAR_0029', 'VAR_0030', 'VAR_0031', 'VAR_0032', 'VAR_0038', 'VAR_0039', 'VAR_0040', 'VAR_0041', 'VAR_0042', 'VAR_0043', 'VAR_0044', 'VAR_0188', 'VAR_0189', 'VAR_0190', 'VAR_0196', 'VAR_0197', 'VAR_0199', 'VAR_0202', 'VAR_0203', 'VAR_0207', 'VAR_0213', 'VAR_0215', 'VAR_0216', 'VAR_0221', 'VAR_0222', 'VAR_0223', 'VAR_0229', 'VAR_0239', 'VAR_0246', 'VAR_0394', 'VAR_0438', 'VAR_0446', 'VAR_0527', 'VAR_0528', 'VAR_0530', 'VAR_0840', 'VAR_0847', 'VAR_1428']


In [7]:
# encode string
strings = data.dtypes == 'object'
strings = strings[strings].index.tolist()


In [8]:
#data.columns[data.dtypes=='object'].tolist()

In [9]:
# encode string
strings = data.dtypes == 'object'
strings = strings[strings].index.tolist()
encoders = {}

for col in strings:
    encoders[col] = preprocessing.LabelEncoder()
    data[col] = encoders[col].fit_transform(data[col])
    try:
        test[col] = encoders[col].transform(test[col])
    except:
        # lazy way to incorporate the feature only if can be encoded in the test set
        del test[col]
        del data[col]
        #data.drop([col], axis = 1, inplace = True)
        #test.drop([col], axis = 1, inplace = True)
        
        
        

/home/ubuntu/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
/home/ubuntu/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:259: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  return aux[:-1][aux[1:] == aux[:-1]]
/home/ubuntu/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:394: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  bool_ar = (sar[1:] != sar[:-1])


In [10]:
print data.shape
print test.shape

(145231, 1858)
(145232, 1857)


In [11]:
# DATA ready
X = data.drop('target',1).fillna(0)
y = data.target

In [12]:
# RF FTW :)
rf = ensemble.RandomForestClassifier(n_jobs=4, n_estimators = 30, random_state = 11)


In [13]:
# CROSS VALIDATE AND PRINT TRAIN AND TEST SCORE
kf = cross_validation.StratifiedKFold(y, n_folds=3, shuffle=True, random_state=11)
trscores, cvscores, times = [], [], []
for itr, icv in kf:
    t = time()
    trscore = auc(y.iloc[itr], rf.fit(X.iloc[itr], y.iloc[itr]).predict_proba(X.iloc[itr])[:,1])
    cvscore = auc(y.iloc[icv], rf.predict_proba(X.iloc[icv])[:,1])
    trscores.append(trscore); cvscores.append(cvscore); times.append(time()-t)
print "TRAIN %.4f | TEST %.4f | TIME %.2fm (1-fold)" % (np.mean(trscores), np.mean(cvscores), np.mean(times)/60)


TRAIN 1.0000 | TEST 0.7407 | TIME 0.40m (1-fold)


In [14]:
# MAKING SUBMISSION
submission = pd.DataFrame(rf.fit(X,y).predict_proba(test.fillna(0))[:,1], index=test.index, columns=['target'])
submission.index.name = 'ID'
submission.to_csv('beat_withrf.csv')

In [15]:
print trscores
print cvscores

[0.99999914942050239, 0.99999931051057589, 0.9999996359993415]
[0.73629430412199792, 0.74200529036285912, 0.74373365190819729]


In [16]:
enc = encoders['VAR_0001']

In [17]:
enc.classes_

array(['H', 'Q', 'R'], dtype=object)

In [19]:
#for d in encoders:
#    print encoders[d].classes_